### Training the Video Classification Model

### This script does - VGG16

In [1]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import pickle

import time


 




C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:
Drive = "C:"
## Define root folder
RootFolder = Drive+"/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles"




In [3]:
trainFrames  = pd.read_csv(RootFolder+"/trainFrames.csv")

trainFrames.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g08_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g08_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g08_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g08_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g08_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [4]:
trainFrames.shape

(973, 3)

In [5]:
# Get the unique values of 'class' column - How many distinct classes
model_class = trainFrames['class'].unique().tolist()
print(model_class)
model_output_size = len(model_class)
print(model_output_size)


['Basketball', 'SoccerPenalty']
2


In [6]:
trainFrames.shape[0]

973

In [7]:
trainFrames['FullPathName'][0]

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting\\VDOFrames/v_Basketball_g08_c01_frame0.jpg'

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import pafy

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import train_image 

In [9]:
train_image = []

# for loop to read and store frames
for i in tqdm(range(trainFrames.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    # three-dimensional array containing the red, green, and blue pixel intensities. 
    #Image has been reformated to be 224-by-224 pixels in size.
    img = image.load_img(trainFrames['FullPathName'][i], target_size=(224,224,3))
    #print(img.shape)
    # converting it to array
    img = image.img_to_array(img)
    
    # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████████| 973/973 [00:02<00:00, 421.78it/s]


(973, 224, 224, 3)

In [10]:
# make sure that the distribution of each class is similar in both training and validation sets. 
# We can use the stratify parameter

# separating the target
y = trainFrames['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [11]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

### Defining the architecture of the video classification model

### VGG16 on ImageNet

https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1


#### Keras provides a set of state-of-the-art deep learning models along with pre-trained weights on ImageNet. These pre-trained models can be used for image classification, feature extraction, and transfer learning

##### VGG16 refers to a VGG model with 16 weight layers, and VGG19 refers to a VGG model with 19 weight layers.  
#####  1.The input layer takes an image in the size of (224 x 224 x 3), and the output layer is a softmax prediction on classes.

#####  2.From the input layer to the last max pooling layer (labeled by 7 x 7 x 512) is regarded as the feature extraction part of the model, 

##### 3 while the rest of the network is regarded as the classification part of the model.

## VGG-16 pre-trained model will be used
### https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

### creating the base model of pre-trained VGG16 model


In [12]:

# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

### set include_top = False it overlooks the last 3 Fully Connected(FC) layers 


base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### vgg16 (without last 3 Fully Connected(FC)) is an impressively large neural network. It has 14,714,688 parameters. 
To see all paratemer use 
vgg19_full = VGG19(weights='imagenet')
vgg19_full.summary()

In [13]:

%%time
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape


Wall time: 13.5 s


(778, 7, 7, 512)

In [14]:
%%time
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape


Wall time: 2.24 s


(195, 7, 7, 512)

In [15]:
print(X_train.shape)
print(X_test.shape)
      
    

(778, 7, 7, 512)
(195, 7, 7, 512)


There are 195 images in the test set and the shape of these images has also changed to (7, 7, 512). We will use a fully connected network now to fine-tune the model. This fully connected network takes input in single dimension. So, we will reshape the images into a single dimension:

In [16]:
X_train = X_train.reshape(778, 7*7*512)
X_test = X_test.reshape(195, 7*7*512)

In [17]:
# normalize the pixel values, i.e., keep the pixel values between 0 and 1. This helps the model to converge faster.

In [18]:
X_train.max()

10.580899

In [19]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [20]:
# shape of images
X_train.shape

(778, 25088)

### Keras Neural Network Sequential Model
##### More Info - https://faroit.com/keras-docs/1.0.1/getting-started/sequential-model-guide/

In [21]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(model_output_size, activation='softmax'))  # model_output_size is 2 - already defined



###### 1 x Dense layer of 1024 units

###### 1 x Dense layer of 512 units
###### 1 x Dense layer of 256 units
###### 1 x Dense layer of 128 units


#### RELU activation for first four dense layers of  units to stop forwarding negative values through the network.

#### 2 unit dense layer in the end with softmax activation as there are 2 classes (model_output_size) to predict from in the end which are Basketball and SoccerPenalty. 
#### The softmax layer will output the value between 0 and 1 based on the confidence of the model that which class the images belongs to.

## Dropout
####   Usually, dropout is placed on the fully connected layers only because they are the one with the greater number of parameters and thus they're likely to excessively co-adapting themselves causing overfitting. However, since it's a stochastic regularization technique, you can really place it everywhere
#####  Dropout here has been applied applied to hidden neurons in the body of network model.
#### a dropout rate of 50% is used 

In [22]:
# Printing the models summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

#### ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

### -----

### save_best_only: if save_best_only=True, it only saves when the model is considered the "best" and the latest best model according to the quantity monitored will not be overwritten

### The metric name to monitormode: one of {'auto', 'min', 'max'}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity. For val_acc, this should be max, for val_loss this should be min

#### mode: one of {'auto', 'min', 'max'}. If save_best_only=True, -- the decision to overwrite the current save file is made based on either the maximization or the minimization  of the monitored quantity. For val_acc, this should be max, for val_loss this should be min

In [23]:
###Training the video classification model

# defining a function to save the weights of best model
### from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')




#### using Adam optimiser 

In [24]:
# compiling the model - Adding loss, optimizer and metrics values to the model.
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])



In [25]:
%%time
# training the model
#model_training_history = model.fit( X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

# Adding Early Stopping Callback
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

model_training_history = model.fit(x = X_train, y = y_train, epochs = 50, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks=[mcp_save])

Train on 622 samples, validate on 156 samples
Epoch 1/50
622/622 [==============================] - ETA: 1:22 - loss: 0.7369 - accuracy: 0.50 - ETA: 30s - loss: 0.7770 - accuracy: 0.6667 - ETA: 20s - loss: 0.7072 - accuracy: 0.650 - ETA: 16s - loss: 0.9313 - accuracy: 0.642 - ETA: 13s - loss: 1.0498 - accuracy: 0.638 - ETA: 11s - loss: 0.9518 - accuracy: 0.659 - ETA: 10s - loss: 1.1837 - accuracy: 0.596 - ETA: 9s - loss: 1.2958 - accuracy: 0.566 - ETA: 9s - loss: 1.2744 - accuracy: 0.55 - ETA: 8s - loss: 1.2440 - accuracy: 0.57 - ETA: 8s - loss: 1.2043 - accuracy: 0.57 - ETA: 7s - loss: 1.1441 - accuracy: 0.58 - ETA: 7s - loss: 1.1633 - accuracy: 0.59 - ETA: 7s - loss: 1.2017 - accuracy: 0.57 - ETA: 6s - loss: 1.1580 - accuracy: 0.57 - ETA: 6s - loss: 1.1716 - accuracy: 0.55 - ETA: 6s - loss: 1.1874 - accuracy: 0.55 - ETA: 6s - loss: 1.2138 - accuracy: 0.54 - ETA: 5s - loss: 1.2584 - accuracy: 0.52 - ETA: 5s - loss: 1.2303 - accuracy: 0.53 - ETA: 5s - loss: 1.2271 - accuracy: 0.53 - ET

622/622 [==============================] - ETA: 5s - loss: 0.0036 - accuracy: 1.00 - ETA: 5s - loss: 0.0119 - accuracy: 1.00 - ETA: 5s - loss: 0.0125 - accuracy: 1.00 - ETA: 5s - loss: 0.0089 - accuracy: 1.00 - ETA: 5s - loss: 0.0072 - accuracy: 1.00 - ETA: 5s - loss: 0.0061 - accuracy: 1.00 - ETA: 5s - loss: 0.0070 - accuracy: 1.00 - ETA: 5s - loss: 0.0066 - accuracy: 1.00 - ETA: 4s - loss: 0.0059 - accuracy: 1.00 - ETA: 4s - loss: 0.0055 - accuracy: 1.00 - ETA: 4s - loss: 0.0051 - accuracy: 1.00 - ETA: 4s - loss: 0.0046 - accuracy: 1.00 - ETA: 4s - loss: 0.0049 - accuracy: 1.00 - ETA: 4s - loss: 0.0049 - accuracy: 1.00 - ETA: 4s - loss: 0.0047 - accuracy: 1.00 - ETA: 4s - loss: 0.0044 - accuracy: 1.00 - ETA: 4s - loss: 0.0042 - accuracy: 1.00 - ETA: 4s - loss: 0.0039 - accuracy: 1.00 - ETA: 4s - loss: 0.0037 - accuracy: 1.00 - ETA: 4s - loss: 0.0036 - accuracy: 1.00 - ETA: 4s - loss: 0.0035 - accuracy: 1.00 - ETA: 4s - loss: 0.0034 - accuracy: 1.00 - ETA: 4s - loss: 0.0032 - accuracy

622/622 [==============================] - ETA: 6s - loss: 0.0249 - accuracy: 1.00 - ETA: 7s - loss: 0.0126 - accuracy: 1.00 - ETA: 7s - loss: 0.0110 - accuracy: 1.00 - ETA: 7s - loss: 0.0083 - accuracy: 1.00 - ETA: 8s - loss: 0.0068 - accuracy: 1.00 - ETA: 8s - loss: 0.0057 - accuracy: 1.00 - ETA: 8s - loss: 0.0071 - accuracy: 1.00 - ETA: 8s - loss: 0.0069 - accuracy: 1.00 - ETA: 10s - loss: 0.0062 - accuracy: 1.000 - ETA: 10s - loss: 0.0057 - accuracy: 1.000 - ETA: 10s - loss: 0.0052 - accuracy: 1.000 - ETA: 9s - loss: 0.0333 - accuracy: 0.982 - ETA: 9s - loss: 0.0311 - accuracy: 0.98 - ETA: 9s - loss: 0.0293 - accuracy: 0.98 - ETA: 9s - loss: 0.0276 - accuracy: 0.98 - ETA: 9s - loss: 0.0261 - accuracy: 0.98 - ETA: 9s - loss: 0.0257 - accuracy: 0.98 - ETA: 9s - loss: 0.0244 - accuracy: 0.98 - ETA: 9s - loss: 0.0239 - accuracy: 0.98 - ETA: 9s - loss: 0.0230 - accuracy: 0.98 - ETA: 9s - loss: 0.0221 - accuracy: 0.98 - ETA: 9s - loss: 0.0211 - accuracy: 0.98 - ETA: 9s - loss: 0.0203 - a

622/622 [==============================] - ETA: 4s - loss: 0.0242 - accuracy: 1.00 - ETA: 5s - loss: 0.0081 - accuracy: 1.00 - ETA: 5s - loss: 0.0145 - accuracy: 1.00 - ETA: 5s - loss: 0.0104 - accuracy: 1.00 - ETA: 5s - loss: 0.0081 - accuracy: 1.00 - ETA: 5s - loss: 0.0066 - accuracy: 1.00 - ETA: 5s - loss: 0.0061 - accuracy: 1.00 - ETA: 5s - loss: 0.0056 - accuracy: 1.00 - ETA: 5s - loss: 0.0052 - accuracy: 1.00 - ETA: 5s - loss: 0.0046 - accuracy: 1.00 - ETA: 5s - loss: 0.0041 - accuracy: 1.00 - ETA: 5s - loss: 0.0037 - accuracy: 1.00 - ETA: 5s - loss: 0.0035 - accuracy: 1.00 - ETA: 5s - loss: 0.0033 - accuracy: 1.00 - ETA: 5s - loss: 0.0031 - accuracy: 1.00 - ETA: 5s - loss: 0.0028 - accuracy: 1.00 - ETA: 5s - loss: 0.0027 - accuracy: 1.00 - ETA: 5s - loss: 0.0025 - accuracy: 1.00 - ETA: 4s - loss: 0.0024 - accuracy: 1.00 - ETA: 4s - loss: 0.0023 - accuracy: 1.00 - ETA: 4s - loss: 0.0021 - accuracy: 1.00 - ETA: 4s - loss: 0.0020 - accuracy: 1.00 - ETA: 4s - loss: 0.0021 - accuracy

622/622 [==============================] - ETA: 4s - loss: 3.0308e-05 - accuracy: 1.00 - ETA: 5s - loss: 0.0040 - accuracy: 1.0000   - ETA: 4s - loss: 0.0025 - accuracy: 1.00 - ETA: 4s - loss: 0.0063 - accuracy: 1.00 - ETA: 5s - loss: 0.0399 - accuracy: 0.97 - ETA: 5s - loss: 0.0348 - accuracy: 0.97 - ETA: 4s - loss: 0.0299 - accuracy: 0.98 - ETA: 4s - loss: 0.0259 - accuracy: 0.98 - ETA: 4s - loss: 0.0230 - accuracy: 0.98 - ETA: 4s - loss: 0.0206 - accuracy: 0.98 - ETA: 4s - loss: 0.0187 - accuracy: 0.98 - ETA: 4s - loss: 0.0172 - accuracy: 0.98 - ETA: 4s - loss: 0.0166 - accuracy: 0.99 - ETA: 4s - loss: 0.0154 - accuracy: 0.99 - ETA: 4s - loss: 0.0144 - accuracy: 0.99 - ETA: 4s - loss: 0.0135 - accuracy: 0.99 - ETA: 4s - loss: 0.0127 - accuracy: 0.99 - ETA: 4s - loss: 0.0120 - accuracy: 0.99 - ETA: 4s - loss: 0.0114 - accuracy: 0.99 - ETA: 4s - loss: 0.0116 - accuracy: 0.99 - ETA: 4s - loss: 0.0113 - accuracy: 0.99 - ETA: 4s - loss: 0.0108 - accuracy: 0.99 - ETA: 4s - loss: 0.0103 - 

...

622/622 [==============================] - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 3.0133e-07 - accuracy: 1.000 - ETA: 18s - loss: 2.7120e-07 - accuracy: 1.000 - ETA: 17s - loss: 2.4655e-07 - accuracy: 1.000 - ETA: 17s - loss: 2.2600e-07 - accuracy: 1.000 - ETA: 17s - loss: 2.0862e-07 - accuracy: 1.000 - ETA: 17s - loss: 1.9371e-07 - accuracy: 1.000 - ETA: 17s - loss: 1.8080e-07 - accuracy: 1.000 - ETA: 17s - loss: 1.6950e-07 - accuracy: 1.000 - ETA: 17s - loss: 1.5953e-07 - accuracy: 1.000 - ETA: 17s - loss: 0.0060 - accuracy: 1.0000    - ETA: 17s - loss: 0.0057 - accuracy: 1.000 - ETA: 16s - loss: 0.0054 - accuracy: 1.000 - ETA: 

622/622 [==============================] - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 19s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 19s - loss: 1.4404e-06 - accuracy: 1.000 - ETA: 19s - loss: 1.0803e-06 - accuracy: 1.000 - ETA: 19s - loss: 3.3321e-04 - accuracy: 1.000 - ETA: 19s - loss: 2.7767e-04 - accuracy: 1.000 - ETA: 19s - loss: 2.3801e-04 - accuracy: 1.000 - ETA: 19s - loss: 2.0830e-04 - accuracy: 1.000 - ETA: 19s - loss: 1.8515e-04 - accuracy: 1.000 - ETA: 19s - loss: 1.6664e-04 - accuracy: 1.000 - ETA: 19s - loss: 1.5149e-04 - accuracy: 1.000 - ETA: 19s - loss: 1.3886e-04 - accuracy: 1.000 - ETA: 19s - loss: 1.2818e-04 - accuracy: 1.000 - ETA: 18s - loss: 1.2492e-04 - accuracy: 1.000 - ETA: 18s - loss: 1.1659e-04 - accuracy: 1.000 - ETA: 18s - loss: 1.1477e-04 - accuracy: 1.000 - ETA: 18s - loss: 0.0086 - accuracy: 1.0000    - ETA: 18s - loss: 0.0081 - accuracy: 1.000 - ETA: 18s - loss: 0.0077 - accuracy: 1.000 - ETA: 18s - loss: 0.0073 - accuracy: 1.000 - ETA: 17s 

622/622 [==============================] - ETA: 19s - loss: 2.9802e-08 - accuracy: 1.000 - ETA: 21s - loss: 1.4901e-08 - accuracy: 1.000 - ETA: 23s - loss: 1.1473e-05 - accuracy: 1.000 - ETA: 23s - loss: 8.6048e-06 - accuracy: 1.000 - ETA: 22s - loss: 6.8839e-06 - accuracy: 1.000 - ETA: 22s - loss: 5.7366e-06 - accuracy: 1.000 - ETA: 22s - loss: 4.9170e-06 - accuracy: 1.000 - ETA: 21s - loss: 4.3024e-06 - accuracy: 1.000 - ETA: 21s - loss: 3.8244e-06 - accuracy: 1.000 - ETA: 21s - loss: 3.4419e-06 - accuracy: 1.000 - ETA: 20s - loss: 3.1372e-06 - accuracy: 1.000 - ETA: 20s - loss: 2.8757e-06 - accuracy: 1.000 - ETA: 20s - loss: 2.9964e-05 - accuracy: 1.000 - ETA: 20s - loss: 2.7824e-05 - accuracy: 1.000 - ETA: 20s - loss: 2.5969e-05 - accuracy: 1.000 - ETA: 19s - loss: 2.4346e-05 - accuracy: 1.000 - ETA: 19s - loss: 2.2914e-05 - accuracy: 1.000 - ETA: 19s - loss: 2.1641e-05 - accuracy: 1.000 - ETA: 19s - loss: 2.0502e-05 - accuracy: 1.000 - ETA: 19s - loss: 1.9476e-05 - accuracy: 1.000

622/622 [==============================] - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 17s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 17s - loss: 4.5850e-09 - accuracy: 1.000 - ETA: 17s - loss: 4.2575e-09 - accuracy: 1.000 - ETA: 17s - loss: 0.0015 - accuracy: 1.0000    - ETA: 17s - loss: 0.0014 - accuracy: 1.000 - ETA: 17s - loss: 0.0013 - accuracy: 1.000 - ETA: 17s - loss: 0.0013 - accuracy: 1.000 - ETA: 17s - loss: 0.0012 - accuracy: 1.000 - ETA: 16s - loss: 0.0011 - accuracy: 1.000 - ETA: 16s - loss: 

622/622 [==============================] - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 0.0000e+00 - accuracy: 1.000 - ETA: 18s - loss: 5.9605e-09 - accuracy: 1.000 - ETA: 18s - loss: 4.8225e-07 - accuracy: 1.000 - ETA: 17s - loss: 4.4206e-07 - accuracy: 1.000 - ETA: 17s - loss: 4.0806e-07 - accuracy: 1.000 - ETA: 17s - loss: 3.7891e-07 - accuracy: 1.000 - ETA: 17s - loss: 3.5365e-07 - accuracy: 1.000 - ETA: 17s - loss: 3.3155e-07 - accuracy: 1.000 - ETA: 17s - loss: 3.1204e-07 - accuracy: 1.000 - ETA: 17s - loss: 9.0094e-04 - accuracy: 1.000 - ETA: 17s - loss: 8.5352e-04 - accuracy: 1.000 - ETA: 16s - loss: 8.1086e-04 - accuracy: 1.000


### Evaluate trained model on the feature’s and label’s test sets.

In [26]:
model_evaluation_history = model.evaluate(X_test, y_test)

195/195 [==============================] - ETA:  - ETA:  - 0s 590us/step


In [27]:
### Save model in a folder 
RootFolder

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles'

In [28]:
# Saving Model
model_name = 'video combined_video-2vdo model_jj.h5'
model.save(RootFolder+'/'+model_name)

### How to load the model from disk

In [29]:
RootFolder

'C:/1-GG/CAP4/EventDetection/1-ExampleSetting/OutputFiles'

In [30]:
model_name = 'video combined_video-2vdo model_jj.h5'



In [31]:
from tensorflow import keras

model = keras.models.load_model(RootFolder+'/'+model_name)


#### Here

### Evaluating our Video Classification Model


In [32]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip


## pafy, youtube-dl and moviepy packages to be installed
##pip install pafy youtube-dl moviepy

## Install open cv - make sure numpy already installed
#pip install opencv-python
#import pafy

In [33]:
# read testFrames csv
testFrames  = pd.read_csv(RootFolder+"/testFrames.csv")
testFrames.shape

(319, 3)

In [34]:
testFrames.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c01.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [35]:
# Extract each vdo from testFrames 
testSetVDOlist = testFrames["FrameFilename"].tolist()
print(testSetVDOlist)

['v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c01.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c02.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c03.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c04.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c05.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c06.avi', 'v_Basketball_g01_c

In [36]:
print(len(testSetVDOlist))

# Remove Duplicates From vdo list - as a vdo file has many frame images
testSetVDOlist = list(dict.fromkeys(testSetVDOlist))
print(len(testSetVDOlist))
 

319
76


In [37]:
testSetVDOlist

['v_Basketball_g01_c01.avi',
 'v_Basketball_g01_c02.avi',
 'v_Basketball_g01_c03.avi',
 'v_Basketball_g01_c04.avi',
 'v_Basketball_g01_c05.avi',
 'v_Basketball_g01_c06.avi',
 'v_Basketball_g01_c07.avi',
 'v_Basketball_g02_c01.avi',
 'v_Basketball_g02_c02.avi',
 'v_Basketball_g02_c03.avi',
 'v_Basketball_g02_c04.avi',
 'v_Basketball_g02_c05.avi',
 'v_Basketball_g02_c06.avi',
 'v_Basketball_g03_c01.avi',
 'v_Basketball_g03_c02.avi',
 'v_Basketball_g03_c03.avi',
 'v_Basketball_g03_c04.avi',
 'v_Basketball_g03_c05.avi',
 'v_Basketball_g03_c06.avi',
 'v_Basketball_g04_c01.avi',
 'v_Basketball_g04_c02.avi',
 'v_Basketball_g04_c03.avi',
 'v_Basketball_g04_c04.avi',
 'v_Basketball_g05_c01.avi',
 'v_Basketball_g05_c02.avi',
 'v_Basketball_g05_c03.avi',
 'v_Basketball_g05_c04.avi',
 'v_Basketball_g06_c01.avi',
 'v_Basketball_g06_c02.avi',
 'v_Basketball_g06_c03.avi',
 'v_Basketball_g06_c04.avi',
 'v_Basketball_g07_c01.avi',
 'v_Basketball_g07_c02.avi',
 'v_Basketball_g07_c03.avi',
 'v_Basketball

In [38]:
# separating the target
# To map the predicted categories with the actual categories, we will use train frame class:
y = trainFrames['class']
y = pd.get_dummies(y)

In [39]:
y

,Basketball,SoccerPenalty
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
968,0,1
969,0,1
970,0,1
971,0,1


### Example

In [40]:
testSetVDOlist[1]

'v_Basketball_g01_c02.avi'

In [41]:
temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[1])]
temp_df.head()

,FrameFilename,FullPathName,class
5,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
6,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
7,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
8,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
9,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [42]:
temp_df = temp_df.reset_index(drop=True)  # reset index
temp_df.head()

,FrameFilename,FullPathName,class
0,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
1,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
2,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
3,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball
4,v_Basketball_g01_c02.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,Basketball


In [43]:
print(temp_df['class'][0])

Basketball


In [44]:

    
for i in range(len(testSetVDOlist)):
    print(testSetVDOlist[i])
    temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[i])]
    temp_df = temp_df.reset_index(drop=True)  # reset index
    print(temp_df['class'][0])
    #print(temp_df.shape)

v_Basketball_g01_c01.avi
Basketball
v_Basketball_g01_c02.avi
Basketball
v_Basketball_g01_c03.avi
Basketball
v_Basketball_g01_c04.avi
Basketball
v_Basketball_g01_c05.avi
Basketball
v_Basketball_g01_c06.avi
Basketball
v_Basketball_g01_c07.avi
Basketball
v_Basketball_g02_c01.avi
Basketball
v_Basketball_g02_c02.avi
Basketball
v_Basketball_g02_c03.avi
Basketball
v_Basketball_g02_c04.avi
Basketball
v_Basketball_g02_c05.avi
Basketball
v_Basketball_g02_c06.avi
Basketball
v_Basketball_g03_c01.avi
Basketball
v_Basketball_g03_c02.avi
Basketball
v_Basketball_g03_c03.avi
Basketball
v_Basketball_g03_c04.avi
Basketball
v_Basketball_g03_c05.avi
Basketball
v_Basketball_g03_c06.avi
Basketball
v_Basketball_g04_c01.avi
Basketball
v_Basketball_g04_c02.avi
Basketball
v_Basketball_g04_c03.avi
Basketball
v_Basketball_g04_c04.avi
Basketball
v_Basketball_g05_c01.avi
Basketball
v_Basketball_g05_c02.avi
Basketball
v_Basketball_g05_c03.avi
Basketball
v_Basketball_g05_c04.avi
Basketball
v_Basketball_g06_c01.avi
Bas

In [45]:
#print(temp_df['class'][i])
temp_df.head()

,FrameFilename,FullPathName,class
0,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
1,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
2,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty
3,v_SoccerPenalty_g07_c06.avi,C:/1-GG/CAP4/EventDetection/1-ExampleSetting\V...,SoccerPenalty


### end of Example

In [46]:
# for loop to read and store test frames
# creating two lists to store predicted and actual tags
predict = []
actual = []
indent_variable   = []

for i in range(len(testSetVDOlist)):
    #print(testSetVDOlist[i])
    temp_df = testFrames.loc[(testFrames.FrameFilename == testSetVDOlist[i])] # all frames of a particular vdo
    temp_df = temp_df.reset_index(drop=True)  # reset index
    #print(temp_df.shape)  
   
    prediction_images = []
    
    # appending the actual class of the video
    
    actual.append(temp_df['class'][0])
    #print("Actual:" , actual)

    # appending the vdo file name as indent
    indent_variable.append(temp_df['FrameFilename'][0])
    #print("Indent:" ,indent_variable)

    for i in tqdm(range(temp_df.shape[0])):  ### Loop through all frames of a particular vdo
        
 
        # loading the image and keeping the target size as (224,224,3)
        img = image.load_img(temp_df['FullPathName'][i], target_size=(224,224,3))
        # converting it to array
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img/255
        # appending the image to the image list
        prediction_images.append(img)
        #print("Prediction:" , prediction_images)
        
    # converting the list to numpy array
    prediction_images = np.array(prediction_images)

    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)

    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)

    # predicting class for each array.
    # predict_classes() function on model in order to predict the class values for each instance in the array.
    prediction = model.predict_classes(prediction_images)
    #print("Prediction:" , prediction)
    # appending the mode of predictions (Class that occurs on highest frrequency) in predict list to assign the class to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 96.16it/s]


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 399.98it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 442.81it/s]


In [47]:
prediction

array([1, 1, 1, 1], dtype=int64)

In [48]:
y.columns.values[s.mode(prediction)[0][0]]

'SoccerPenalty'

In [49]:
# Calling DataFrame constructor after zipping
# both lists, with columns specified
EvalFinalOutput = pd.DataFrame(list(zip(indent_variable, actual ,predict )),
               columns =['VdOFileName', 'ActualClass' , 'PredictedClass'])
EvalFinalOutput

,VdOFileName,ActualClass,PredictedClass
0,v_Basketball_g01_c01.avi,Basketball,Basketball
1,v_Basketball_g01_c02.avi,Basketball,Basketball
2,v_Basketball_g01_c03.avi,Basketball,Basketball
3,v_Basketball_g01_c04.avi,Basketball,Basketball
4,v_Basketball_g01_c05.avi,Basketball,Basketball
...,...,...,...
71,v_SoccerPenalty_g07_c02.avi,SoccerPenalty,SoccerPenalty
72,v_SoccerPenalty_g07_c03.avi,SoccerPenalty,SoccerPenalty
73,v_SoccerPenalty_g07_c04.avi,SoccerPenalty,SoccerPenalty
74,v_SoccerPenalty_g07_c05.avi,SoccerPenalty,SoccerPenalty


In [50]:
EvalFinalOutput.to_csv(RootFolder+'/EvalFinalOutput.csv',header=True, index=False)